THIS IS THE NOTEBEOOK WHERE WE ARE GOING TO FIRST INITIALIZE A MODEL AND THEN THEN EMBEDD THE DATA INTO THE RAG PIPELINE


In [3]:
import pandas as pd
from langchain_core.documents import Document

# Load the FIFA matches data
df = pd.read_csv("data/Fifa_world_cup_matches.csv")  # adjust path if needed

# Quick peek
print(df.head())
print(df.columns)


           team1         team2 possession team1 possession team2  \
0          QATAR       ECUADOR              42%              50%   
1        ENGLAND          IRAN              72%              19%   
2        SENEGAL   NETHERLANDS              44%              45%   
3  UNITED STATES         WALES              51%              39%   
4      ARGENTINA  SAUDI ARABIA              64%              24%   

  possession in contest  number of goals team1  number of goals team2  \
0                    8%                      0                      2   
1                    9%                      6                      2   
2                   11%                      0                      2   
3                   10%                      1                      1   
4                   12%                      1                      2   

          date     hour category  ...  penalties scored team1  \
0  20 NOV 2022  17 : 00  Group A  ...                       0   
1  21 NOV 2022  14 : 0

converting each row of the data into text to embedded into the vector Store

In [ ]:
df.columns


In [ ]:
df.columns

In [4]:
def row_to_text(row):
    # Adjust column names to your actual CSV headers
    return (
       
        f"FIFA World Cup 2022 Match {row['team1']} vs {row['team2']}. "
        f"Score: {row['number of goals team1']} - {row['number of goals team2']}. "
        f"Date: {row['date']}. "
        f"Category:{row['category']} ,"
        f"Possesion of {row['team1']} is {row['possession team1']} and {row['team2']} is {row['possession team2']} "
        f"{row['team1']} had {row['on target attempts team1']} On Target Attempts and {row['team2']} had  {row['on target attempts team2']} attempts"
         f"{row['team1']} had {row['assists team1']} assists and {row['team2']} had {row['assists team2']} assists"
    )

docs = []
for _, row in df.iterrows():
    text = row_to_text(row)
    metadata = {
        "team1": row["team1"],
        "team2": row["team2"],
        "category":row['category']
    }
    docs.append(Document(page_content=text, metadata=metadata))

len(docs), docs[0]


(64,
 Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. Category:Group A ,Possesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'))

In [ ]:
print(docs)

INTIATING THE EMBEDDING MODEL

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)



TEXT SPLITTER FOR THE RECAPS DATA


Vector Storing

In [6]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embedding=embeddings)


Now that the vector embeddings are completed we will now be doing a test for similarity search so that we can see what is actually going on here now using similiratiy_score_threshhold

similarity = “just give me closest stuff”

similarity_score_threshold = “closest stuff but drop trash”

mmr = “closest but also diverse (good for longer answers)”

In [7]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 10,                 # consider up to 10
        "score_threshold": 0.9   # filter out low similarity
    }
)



In [8]:
retriever_s = vector_store.as_retriever(
    search_type="similarity",               # or "similarity_score_threshold", "mmr"
    search_kwargs={"k": 5}                  # top 5 matches
)


In [ ]:
prompt = "Tell me about the match between Portugal and Morocco"  # ⬅️ Add this

results = retriever_s.invoke(prompt)   # ⬅️ this replaces get_relevant_documents

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 40)

Now Setting Up The Actualy LLM and then seeing what can be done

In [9]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool,ToolRuntime

In [10]:
llm = ChatOllama(
    model="llama3.2:3b",  # or any local ollama model name you’ve pulled
    temperature=0.2,
)

checkpointer = InMemorySaver()

In [11]:

from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(retriever_s,name='retriever_data',description='Usign this tool we will retrive the data from the vector store')



In [12]:
@tool
def retrieve_data(query: str) -> str:
    """Retrieve World Cup match data from the vector store"""
    results = retriever_s.invoke(query)
    return "\n\n".join([doc.page_content for doc in results])

checkpointer = InMemorySaver()

agent = create_agent(
    model=llm,
    tools=[retrieve_data],
    checkpointer=checkpointer,
    system_prompt=""""""
)

In [13]:
config={"configurable": {"thread_id": "9"}}

result=agent.invoke({
    "messages":[{"role":"user","content":"Hey I am Ananda Prem How are you doing"}]
},config)

resulted=agent.invoke({
    "messages":[{"role":"user","content":"How Did Urugay Perform in this World Cup"}]
},config)

result_2=agent.invoke({
    "messages":[{"role":"user","content":"What would you say about the key moment where their fate in the worldcup was decided"}]
},config)



In [14]:
print("Result 1:")
print(resulted["messages"][-1].content)

print("\nResult 2:")
print(result_2["messages"][-1].content)

print("\nResult 3:")
print(result["messages"][-1].content)

Result 1:
Ananda Prem, it seems that Uruguay's performance in the FIFA World Cup 2022 was not as strong as expected. In their first match against Korea Republic, they were unable to score any goals, but managed to hold their own with a possession of 49% compared to Korea Republic's 38%. However, they had no on-target attempts and no assists.

In their second match against Portugal, Uruguay was on the receiving end of a 2-0 loss. Despite having a possession of 35%, they were unable to overcome Portugal's strong team. They also had only 3 on-target attempts, which was not enough to secure a win.

Uruguay's third and final group stage match was against Ghana, which ended in a 2-0 loss for Uruguay. They had a possession of 39% compared to Ghana's 46%, but were unable to overcome the Ghanaian team. They also had only 7 on-target attempts, which was not enough to secure a win.

Uruguay's performance in the knockout stage is not available in the data I have access to. However, it's worth noti

In [ ]:
print("Result 3 (Full):")
print(result_3["messages"][-1].content)


In [ ]:
result["messages"][-1].pretty_print()
result_2["messages"][-1].pretty_print()
result_3["messages"][-1].pretty_print()


In [ ]:
config={"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is Nathan"}, config)
resp=agent.invoke({"messages": "Tell me about Portugals Match Against Morroco"}, config)
resp=agent.invoke({"messages": "What Would You tell About the possession did portugal deserve to win based off the possesion they maintained throught the match"}, config)
resp["messages"][-1].pretty_print()
resp=agent.invoke({"messages":"What is my name again ?"},config)
resp["messages"][-1].pretty_print()


In [ ]:
config={"configurable": {"thread_id": "1"}}
resp=agent.invoke({"messages":"Tell me about Portugals Match Against Morroco"},config)
resp["messages"][-1].pretty_print()

Creating a Function to Get Input from a User instead Of hardCoding it

its now inside main_func.py from where we are going to call it

In [16]:
# Test with new thread and different question
print(ask_fifa_bot("Tell me about Brazil vs South Korea match", "new_user_123"))
print("\n" + "="*60 + "\n")
print(ask_fifa_bot("What were the possession stats for Japan vs Croatia?", "new_user_456"))

Here's a summary of the Brazil vs South Korea match:

The match between Brazil and South Korea took place on December 5, 2022, as part of the FIFA World Cup 2022. Brazil won the match with a score of 4-1.

Brazil had more possession of the ball, with 47% compared to South Korea's 44%. Brazil also had more on-target attempts, with 9 compared to South Korea's 6.

The match saw three assists from Brazil, while South Korea did not have any assists. The win marked a significant victory for Brazil in their World Cup campaign.


Based on the tool call response, here are the possession stats for Japan vs Croatia:

* Possession of Japan: 35%
* Possession of Croatia: 51%

These stats were obtained from the FIFA World Cup 2022 Match between Japan and Croatia.
Based on the tool call response, here are the possession stats for Japan vs Croatia:

* Possession of Japan: 35%
* Possession of Croatia: 51%

These stats were obtained from the FIFA World Cup 2022 Match between Japan and Croatia.


In [ ]:
print(ask_fifa_bot("Tell me about Brazil vs South Korea match", "new_user_023"))

HElllo